<a href="https://colab.research.google.com/github/ckim415/OOOA/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_by_%ED%83%80%EC%9D%B4%ED%8B%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 코렙에서 한글 사용

In [ ]:
!apt -qq -y install fonts-nanum
 
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
 
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
fm._rebuild()
 
# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'
 
# Colab 의 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic') 

fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


# 구글드라이브 연동

In [2]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# 구글 드라이브 파일 확인
!ls '/gdrive/My Drive/프로젝트2'

# 반복되는 드라이브 경로 변수화
drive_path = '/gdrive/My Drive/프로젝트2/'

Mounted at /gdrive
 6144_link.csv		       game_df.csv
'추천시스템 by 타이틀.ipynb'   home.csv
 concat_df.csv		      '프로젝트 데이터 분석.ipynb'
 crawling_final.ipynb	       Untitled2.ipynb
 edu_link_df


In [3]:
import pandas as pd
import numpy as np

# 데이터 가져오기

In [4]:
df = pd.read_csv(drive_path + 'concat_df.csv')
df.head()

,Unnamed: 0,categories,title,company_names,target_amounts,funding_amounts,percentages,n_supporters,likes,durations,start,end,time,descriptions,result
0,0,뷰티,[와디즈 최초]20대 딸이랑 50대 엄마랑 함께 쓰는 주름개선 만능 크림,주식회사칼라코드,500000,2338880,467,54,41,"['2020.02.28', '2020.03.06']",2020-02-28,2020-03-06,7,"테라젠의 첫 번째 제품, 20대 딸이랑 50대 엄마가 함께 쓰는 주름개선 만능 크림...",1
1,1,뷰티,[2.8억 앵콜] 하루만에 수분140%증가! 임상검증 완료! 갈바닉마스크,반디비타 주식회사,1000000,14260000,1426,145,79,"['2021.04.26', '2021.05.31']",2021-04-26,2021-05-31,35,지금 쓰는 스킨케어 제품의 수분증발을 잡아주고 유효성분은 갈바닉으로 피부 깊숙히 전...,1
2,2,뷰티,1000개 유명 피부과에 공급되는 시크릿 포뮬러를 담은 33패드 프로그램,(주)코떼랑,500000,5017500,1003,65,65,"['2019.04.16', '2019.05.06']",2019-04-16,2019-05-06,20,위시포뮬러가 피부과 화장품 제조 20년의 기술과 노하우를 단 한 장의 패드에 담았...,1
3,3,뷰티,두피 관리의 핵심은 유분량의 균형입니다! 올솔 모닝-나이트 밸런스 샴푸,올솔 (allsol),500000,7358800,1471,134,130,"['2020.03.13', '2020.04.06']",2020-03-13,2020-04-06,24,매일 변하는 두피 유분량! \n그런데도 매번 같은 타입의 샴푸 쓰세요? \n두피뿐만...,1
4,4,뷰티,컨디셔너 필요없이 그린빈샴푸 하나로 두피와 모발의 건강을 책임집니다.,주라이프 네츄럴 코스메틱 주앤솝,500000,694800,138,30,12,"['2019.04.26', '2019.05.08']",2019-04-26,2019-05-08,12,"화학 성분 NO! 모발에 윤기와 탄력을 주며, 비듬이나 가려움을 덜어주며, 컨디셔너...",1


# text 처리

*   특수문자 제거




In [ ]:
import re
from string import punctuation

In [ ]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
!pip install konlpy wordcloud

In [ ]:
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Okt

In [ ]:
kkma = Kkma()
han = Hannanum()
kor = Komoran()
okt = Okt()
# mecab = Mecab()

In [ ]:
#punctutation 제거
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))
#숫자 제거
pattern2 = re.compile(r'[0-9]')

In [ ]:
for i in df['title'][15:30]:
  print(i)
  sentence = pattern1.sub(' ',
             pattern2.sub('',i)).strip()
  # print('kkma :',kkma.nouns(sentence))
  print('han :',han.nouns(sentence))
  print('okt :',okt.nouns(sentence))
  # print('komoran :',kor.nouns(sentence))
  #print('mecab :',Mecab.nouns(sentence))

  print('-'*10)

피부관리는 클렌징으로부터ㅣ하나로 끝낸다ㅣ스피나 미라클 듀얼 퍼펙트 클린저
han : ['피부관리', '클렌징으로부터ㅣ하나', '끝낸다ㅣ스피', '미라클', '듀얼', '퍼펙트', '클린저']
okt : ['피부관리', '클렌징', '하나로', '스피', '미라클', '듀얼', '퍼펙트', '클린', '저']
----------
[1914% 앵콜]후기로 입증된 저분자 히알루론산20000ppm의 흡수력
han : ['앵콜', '후', '입증', '저분자', '히알루론산ppm', '흡수력']
okt : ['앵콜', '후기', '입증', '저분자', '히알루론산', '의', '흡수']
----------
유해 물질로부터 내 아이의 피부를 지켜줄 좋은 습관 만들기, 유아클렌징폼
han : ['물질', '내', '아이', '피부', '습관', '유아클렌징폼']
okt : ['유해', '물질', '로부터', '내', '아이', '피부', '습관', '만들기', '유아', '클렌', '징폼']
----------
 <단독 런칭> 최악의 자외선에서 살아남는 호주사람 99%가 아는 선크림
han : ['단독', '런칭', '최악', '자외선', '호주사람', '선크']
okt : ['단독', '런칭', '최악', '자외선', '호주', '사람', '선크림']
----------
[한방 함유량 20%!!!] 딸바보 아빠가 만든 프라그마 여성 청결제
han : ['한방', '함유량', '딸바보', '아빠', '프라그마', '여성', '청결제']
okt : ['방', '함유량', '딸바보', '아빠', '프라그', '마', '여성', '청', '결제']
----------
헐리우드스타에게 브러쉬만들어주던 언니가 만든 메이크업브러쉬 [1+1+1]
han : ['헐리우드스타에', '브러쉬만들어주', '언니', '메이크업브러쉬']
okt : ['헐리우드', '스타', '브러쉬', '언니', '메이크업', '브러쉬']
----------
'홍보대사 하고 싶을 만큼 최고입니다' 극찬의 후기로

In [ ]:
#함수화
def cleaning(doc):

  sentence = pattern1.sub(' ',
             pattern2.sub('',str(doc))).strip()

  try : 
    return han.nouns(sentence)
  except:
    return []

In [ ]:
df['title_noun'] = df['title'].apply(cleaning)
df['title_noun']

0               [와디즈, 최초, 대, 딸, 대, 엄마, 주름개선, 만능, 크림]
1              [억, 앵콜, 하루만, 수분, 증, 임상검증, 완료, 갈바닉마스크]
2               [개, 유명, 피부과, 공급, 시크릿, 포뮬러, 패드, 프로그램]
3        [두피, 관리, 핵심, 유분량, 균형, 올솔, 모닝, 나이트, 밸런스, 샴푸]
4                  [컨디셔너, 필요, 그린빈샴푸, 하나, 두피, 모발, 건강]
                            ...                     
21475                        [히트패드, 출근길, 핸들, 전기, 핫팩]
21476         [무선, 에어펌프, 자동차, 오토바이공기압, 자전거, 튜브, 축구공]
21477         [너, 나, 간단, 개국어지원, 반응형, 웹페이지, 제작, 실크로드]
21478                        [웨어러블, 공기청정기, 전동식, 마스크]
21479                                      [년JD, 시계]
Name: title_noun, Length: 21480, dtype: object

In [ ]:
df['title_noun'] = df['title_noun'].apply(lambda x: (' ').join(x))

In [ ]:
df['title_noun']

0              와디즈 최초 대 딸 대 엄마 주름개선 만능 크림
1            억 앵콜 하루만 수분 증 임상검증 완료 갈바닉마스크
2             개 유명 피부과 공급 시크릿 포뮬러 패드 프로그램
3        두피 관리 핵심 유분량 균형 올솔 모닝 나이트 밸런스 샴푸
4               컨디셔너 필요 그린빈샴푸 하나 두피 모발 건강
                       ...               
21475                   히트패드 출근길 핸들 전기 핫팩
21476      무선 에어펌프 자동차 오토바이공기압 자전거 튜브 축구공
21477       너 나 간단 개국어지원 반응형 웹페이지 제작 실크로드
21478                  웨어러블 공기청정기 전동식 마스크
21479                              년JD 시계
Name: title_noun, Length: 21480, dtype: object

In [ ]:
df['title_noun'][0].split()[:] == '엄마'

False

## key word rank
- 타이틀 마다 키워드 추출

In [1]:
!pip install krwordrank

In [5]:
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize

In [13]:
example = df['title'][:100].values
example_text = [normalize(text, english=True, number=True) for text in example]

In [14]:
wordrank_extractor = KRWordRank(
    min_count = 2, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10, # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

keywords, rank, graph = wordrank_extractor.extract(example_text, beta, max_iter)

scan vocabs ... 
num vocabs = 548
done


In [15]:
keywords

{'00': 1.0,
 '100': 0.8430677046392853,
 '1억': 1.2022996526970964,
 '20': 1.3881348411880445,
 '30': 1.1228450064259414,
 '50': 0.8097456495402746,
 '90': 0.7418130607503539,
 '9앵콜': 0.9882129634585639,
 '각질': 0.8198846793523364,
 '갈바닉': 1.3285427709559525,
 '강력': 1.0654412326049938,
 '건강': 0.48681744417639466,
 '고순도': 1.749839606258159,
 '관리': 1.368176960217609,
 '기능성': 0.9384100092083218,
 '남성': 0.4177042755373556,
 '닥터': 0.3757040240764192,
 '달라지는': 0.9452757908399642,
 '담은': 1.1105762713918454,
 '당신': 1.447458250161179,
 '두피': 1.1925656142643217,
 '들어간': 1.107395902534647,
 '리얼': 1.2154858573727543,
 '마세요': 1.1704510616314288,
 '마스크': 2.40004623606754,
 '만든': 3.0049019928336262,
 '만들': 0.5147391959372616,
 '명품': 0.9913106502462633,
 '모공': 2.0711880366971362,
 '미백': 1.7112529201743145,
 '밀키': 1.0,
 '바르': 0.6810568504673303,
 '바쁜': 0.5752113580292175,
 '방법': 0.6959831185125956,
 '보습': 1.7722837344156188,
 '비밀': 0.6740123379005033,
 '샴푸': 1.3809277512959426,
 '선크림': 3.406965317553528,

In [16]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
    print('%8s:\t%.4f' % (word, r))

      피부:	6.8477
      앵콜:	6.8291
     하나로:	3.6868
     화장품:	3.4267
     선크림:	3.4070
      만든:	3.0049
     마스크:	2.4000
      여성:	2.1187
      모공:	2.0712
      증가:	1.9784
      좋은:	1.8921
      퍼퓸:	1.8672
      함께:	1.8077
      보습:	1.7723
     고순도:	1.7498
      미백:	1.7113
     콜라겐:	1.6691
    프로그램:	1.6317
      있다:	1.6157
     신개념:	1.6125
    하루만에:	1.5893
      시작:	1.5186
      위한:	1.4932
   수분140:	1.4782
    주름개선:	1.4665
      완료:	1.4620
      당신:	1.4475
      임상:	1.4072
    스킨케어:	1.4046
      탄력:	1.4038


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
title_matrix = count_vect.fit_transform( df['title_noun'] )
title_matrix.shape

(21480, 110708)

In [ ]:
title_sim = cosine_similarity(title_matrix,title_matrix)
title_sim.shape

(21480, 21480)

In [ ]:
np.sort(title_sim)[:, ::-1]

array([[1.        , 0.34188173, 0.34188173, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.27272727, 0.27272727, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.13453456, 0.12403473, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.15569979, 0.13483997, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.56980288, 0.37796447, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.33333333, 0.32025631, ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
title_sim_sorted_ind = title_sim.argsort()[:, ::-1]
title_sim_sorted_ind[1]

array([    1,    24,  1919, ..., 13872, 13873,     0])

In [ ]:
def find_sim_title(df, sorted_ind, title_name, top_n=10):
  title_movie = df[ df['title_noun'].split() == title_name ]
  title_index = title_movie.index.values
  sim_indexs = sorted_ind[title_index, :(top_n)]
  sim_indexs = sim_indexs.reshape(-1)
  return df.iloc[sim_indexs]

#